# Import libraries

In [2]:
pip install ortools

  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.7
    Uninstalling protobuf-4.25.7:
      Successfully uninstalled protobuf-4.25.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.16.2 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import data

In [ ]:
repo_url = 'https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'
full_data = pd.read_csv(repo_url + 'full_data.csv')
distance_matrix = pd.read_csv(repo_url + 'distance_matrix.csv')

In [8]:
full_data

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,N_lv03,lat,lng,distance_to_venoge_km,wed_tickets,thu_tickets,fri_tickets,sat_tickets,sub_tickets,all_tickets
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,161298.778,46.599406,6.540556,0.74,220.0,240.0,225.0,262.0,59.0,770.0
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,162919.512,46.613915,6.531495,2.72,212.0,180.0,179.0,171.0,29.0,655.0
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,162344.417,46.608998,6.564397,3.51,33.0,18.0,15.0,30.0,1.0,93.0
3,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,164336.340,46.626796,6.548602,3.98,66.0,56.0,41.0,63.0,8.0,202.0
4,Sullens,1036,Sullens,5501,VD,2533207.010,1160372.401,fr,1225,12,...,160372.401,46.591278,6.567058,4.12,39.0,54.0,36.0,34.0,4.0,151.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,La Forclaz VD,1866,Ollon,5409,VD,2569188.595,1131844.374,fr,8222,52,...,131844.374,46.337281,7.038465,72.05,1.0,NaN,NaN,NaN,0.0,1.0
235,La Comballaz,1862,Ormont-Dessous,5410,VD,2572060.524,1136176.786,fr,1244,8,...,136176.786,46.376379,7.075504,72.24,NaN,4.0,NaN,NaN,0.0,4.0
236,Rougemont,1659,Château-d'Oex,5841,VD,2580115.427,1147183.777,fr,3656,36,...,147183.777,46.475680,7.179722,74.02,5.0,NaN,NaN,NaN,0.0,5.0
237,Gryon,1882,Gryon,5405,VD,2573628.043,1126382.007,fr,1525,10,...,126382.007,46.288333,7.096432,77.00,3.0,4.0,NaN,5.0,0.0,12.0


In [11]:
full_data[['wed_tickets','thu_tickets','fri_tickets','sat_tickets']].sum()

wed_tickets    8638.0
thu_tickets    7554.0
fri_tickets    4991.0
sat_tickets    6307.0
dtype: float64

In [15]:
demands_wed =  full_data['wed_tickets'].values.tolist()
demands_thu =  full_data['thu_tickets'].values.tolist()
demands_fri =  full_data['fri_tickets'].values.tolist()
demands_sat =  full_data['sat_tickets'].values.tolist()

# Draft model

In [ ]:
# Filter out cities with zero demand
def reduce_problem(distance_matrix, demands):
    indices_to_keep = [i for i, d in enumerate(demands) if d > 0 or i == 0]  # keep depot too
    reduced_matrix = distance_matrix[np.ix_(indices_to_keep, indices_to_keep)]
    reduced_demands = [demands[i] for i in indices_to_keep]
    return reduced_matrix, reduced_demands, indices_to_keep


In [ ]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import numpy as np

def solve_cvrp(distance_matrix, demands, vehicle_capacities, depot=0):
    """Solves the CVRP for one day using OR-Tools."""
    num_vehicles = len(vehicle_capacities)
    num_locations = len(distance_matrix)

    # 1. Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(num_locations, num_vehicles, depot)

    # 2. Create the Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # 3. Distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(distance_matrix[from_node][to_node])  # Must return int

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 4. Demand callback
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return int(demands[from_node])

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        vehicle_capacities,  # list of vehicle capacities
        True,  # start cumul to zero
        'Capacity'
    )

    # 5. Define search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # 6. Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    # 7. Extract and return routes
    if solution:
        routes = []
        for vehicle_id in range(num_vehicles):
            index = routing.Start(vehicle_id)
            route = []
            route_load = 0
            while not routing.IsEnd(index):
                node = manager.IndexToNode(index)
                route.append(node)
                route_load += demands[node]
                index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))  # add end depot
            routes.append((route, route_load))
        return routes
    else:
        print("No solution found!")
        return None
